## ch08
- https://github.com/thampiman/interpretable-ai-book/blob/master/Chapter_08/chapter_08_fairness.ipynb

<div style="text-align: right"> <b>Author : Kwang Myung Yu</b></div>
<div style="text-align: right"> Initial upload: 2023.10.05</div>
<div style="text-align: right"> Last update: 2023.10.05</div>

In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy import stats
import warnings; warnings.filterwarnings('ignore')
#plt.style.use('ggplot')
plt.style.use('seaborn-whitegrid')
%matplotlib inline

In [2]:
from tqdm.notebook import tqdm

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


In [3]:
from sklearn.model_selection import train_test_split
import shap

import pickle 

from tqdm.notebook import tqdm

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


이전 섹션에서는 랜덤 포레스트 모델을 학습시켜 급여를 예측했는데,  
이 모델의 목적은 각 성인이 5만 달러 이상의 소득을 올릴지 여부라는 이분법적 결과를 결정하는 것이었습니다.  

하지만 이러한 예측이 성별, 인종 등 다양한 보호 대상 그룹에 대해 공정하게 이루어졌을까요?   
공정성에 대한 다양한 개념의 정의를 공식화하기 위해 이 모델에 의한 예측과 공정성에 필요한 관련 측정값에 대한 간단한 그림을 살펴봅시다.   
그림 8.10은 2차원 평면에 투영된 모델에 의한 예측을 보여주는 그림입니다.  
랜덤 포레스트 모델은 2차원 평면을 두 개의 반으로 분할하여 긍정적인 예측(오른쪽 반)과 부정적인 예측(왼쪽 반)을 구분합니다.  
성인 20명의 행동 라벨도 이 2-D 평면에 투영되었습니다.  
2-D 평면에서 실제 라벨의 위치는 중요하지 않습니다.  
중요한 것은 라벨이 왼쪽 절반(모델이 음수, 즉 0을 예측하는 경우)에 있는지 오른쪽 절반(모델이 양수, 즉 1을 예측하는 경우)에 있는지의 여부입니다.  
실제 양수 레이블은 원으로 표시되고 실제 음수 레이블은 삼각형으로 표시됩니다.

![Alt text](image-7.png)

- 실제 양수 레이블 - 데이터 집합의 기준값 레이블이 양수인 데이터 포인트입니다. 그림 8.10에서는 데이터 집합에서 연간 소득이 5만 달러 이상인 성인이 원으로 표시되어 있습니다. 원을 세어보면 실제 양성 레이블의 수는 12개입니다.

- 실제 음수 레이블 - 데이터 집합의 기준값 레이블이 음수인 데이터 포인트입니다. 그림 8.10에서는 데이터 집합에서 연간 소득이 5만 달러 이하인 성인이 삼각형으로 표시되어 있습니다. 따라서 실제 음수 레이블의 수는 8개입니다.

- 예측된 양수 - 모델이 긍정적인 결과를 예측하는 데이터 포인트 그림 8.10에서 2차원 평면의 오른쪽 절반에 속하는 데이터 포인트는 양수 예측을 합니다. 해당 영역에 속하는 데이터 포인트는 10개입니다. 따라서 예측된 양성 측정값은 10입니다.

- 예측된 음수 - 모델이 음수 결과를 예측하는 데이터 포인트로, 그림 8.10에서는 2-D 평면의 왼쪽 절반에 속하는 포인트이며 예측된 음수 측정값도 10입니다.

- 진양성 - 그림 8.10에서 진양성은 2-D 평면의 오른쪽 절반에 해당하는 원입니다. 이는 본질적으로 모델이 양수로 예측하는 데이터 포인트이며 실제 레이블도 양수입니다. 이러한 원은 8개가 있으므로 진양성의 수는 8개입니다. 혼동 행렬에서도 이를 구할 수 있는데, 여기서 진양성은 모델이 1을 예측하고 실제 레이블이 1인 경우입니다.

- true negative : 반면에 진정한 네거티브는 2D 평면의 왼쪽 절반에 해당하는 삼각형입니다. 모델이 음수로 예측하고 실제 레이블도 음수인 데이터 포인트입니다. 그림 8.10에서 실제 네거티브의 수는 6입니다. 혼동 행렬에서 모델이 0을 예측하고 실제 레이블이 0인 경우를 볼 수 있습니다.   

- false positive - 오탐은 그림 8.10에서 평면의 오른쪽 절반에 해당하는 삼각형입니다. 모델이 양수로 예측하지만 실제 레이블은 음수인 데이터 포인트입니다. 그림에서 오탐의 수는 2입니다. 혼동 행렬에서 모델은 1을 예측하지만 실제 레이블은 0인 경우입니다.

- false negative - 오탐은 2차원 평면의 왼쪽 절반에 해당하는 원입니다. 기본적으로 모델이 음수로 예측하지만 실제 레이블은 양수인 데이터 포인트입니다. 그림 8.10의 왼쪽 절반에 4개의 원이 있으므로 오탐의 수는 4입니다. 혼동 행렬에서 보면 모델은 0을 예측하지만 실제 레이블은 1인 경우입니다.

Demographic parity